What song characteristics are correlated with higher streaming count?

KL Divergence
Sentiment Analysis

bing lu- modern review data?

what are the words contributing to the sentiment?

Bing Dictionary?

Szyuyet? Trust the result- edit the lexicon?


LDA
Aggregate all of them
Kanye West
Childish Gambino
Frank Ocean

In [1]:
from collections import Counter
import math, glob, csv, re, numpy
from matplotlib import pyplot
import json
from collections import defaultdict
import os
import fnmatch
from datetime import datetime


In [2]:
def find_all(name, path):
    l={}
    for filename in os.listdir(path):
#         print (filename)
        if fnmatch.fnmatch(filename,"*"+name+"*"):
#             print (filename)
#             print (load_album("../"+filename)[0]['release_date'])
            with open("../male_lyrics/"+filename) as json_file:
                data = json.load(json_file)
#                 print (data)
                date = datetime.strptime(data["songs"][0]['year'], '%Y-%M-%d')
                l[filename]=date
#     print (l)
    #     sorted_songs_artist = sorted(l, key=lambda x: x["release_date"])    
    l={k: v for k, v in sorted(l.items(), key=lambda item: item[1])}
#     print (l)
#     print (list(l.keys()))
    sortedl=list(l.keys())
    return sortedl

In [3]:
filenames=find_all("Tyler,","../male_lyrics")


In [4]:
def songkl(artist):
    
    filenames=find_all(artist,"../male_lyrics")
    
    word_pattern = re.compile("\w[\w\-\']*\w|\w")

    

    
   
#     all_counts = Counter()
    songs = []
    for file in filenames:
        
        

        line_buffer = []
        with open("../male_lyrics/"+file) as json_file:
            data = json.load(json_file)

            for row in data["songs"]:
#                 print (row["title"])

        #     for row in data:
#                 print (row)
        #         print("Reading {} ({})".format(row["title"], row["album"]))
                songs.append(row)
#                 print (row)
        #         row["Year"] = int(row["Year"])
                
                counter = Counter()
                row["Tokens"] = counter

                
        #         print ((row['lyrics']))
                line_buffer.append(row['lyrics'])
        #         print (line_buffer)
                counter.update(word_pattern.findall(" ".join(line_buffer)))
#                 print (counter)
                line_buffer = []
#         all_counts += counter
#     print (all_counts)
        # Construct a fixed vocabulary
    # print (all_counts.most_common())
    
    songtokens=defaultdict(dict)
    for song in songs:
#         print (song)
#         print (song["Tokens"])
#     print (type(albumlist[album]))
        songtokens[song['title']]["album"]=song['album']
        date=datetime.strptime(song["year"], '%Y-%M-%d')
        
        songtokens[song['title']]["Release"]=date
        songtokens[song['title']]["Tokens"]=Counter()
    
#         songtokens[song['title']]["Album"]=Counter()
#         for lyric in songs[0]:
#             print (lyric)
        songtokens[song['title']]["Tokens"].update(song["Tokens"])
#     print (songtokens["Start"])
#     print (songtokens)
    check=Counter()
    for i in songtokens:
#         print (i)
#         print ((songtokens[i]["Tokens"]))
        check.update(songtokens[i]["Tokens"])    
#     print (check)
#     print (check['Frisk'])

    vocabulary = [w for w, c in check.most_common()]
#     # print (vocabulary)
    vocabulary_size = len(vocabulary)
#     print (vocabulary_size)
    reverse_vocab = { w: i for i, w in enumerate(vocabulary) }
    
    
#     # Convert counters to vectors
    all_vector = counter_to_vector(check,vocabulary_size,reverse_vocab)
# #     print (all_vector)
# #     print (type(songtokens))
    for song in songtokens:
#         print (songtokens[song]["Tokens"])
        songtokens[song]["Vector"] = counter_to_vector(songtokens[song]["Tokens"],vocabulary_size,reverse_vocab)

#     print ((songtokens["Goblin"]))
#     print ((songtokens))
#     sorted_songs_artist = sorted(songtokens.values(), key=lambda x: x["Release"])
#     print (songtokens)
#     print (sorted_songs_artist)
    for song in songtokens:
#         print (song)
    # `nonzero` returns a tuple with one element per dimension of the input array
        nzs = numpy.nonzero(songtokens[song]["Vector"])[0]
        kl_terms =  songtokens[song]["Vector"][nzs] * numpy.log2(songtokens[song]["Vector"] [nzs] / all_vector[nzs])

        vocabulary_subset = [vocabulary[i] for i in nzs]

        sorted_words = sorted(zip(kl_terms, vocabulary_subset), reverse=True)
        print("Song:",song)
        print ("Album:",songtokens[song]['album'])
        print(", ".join(["{} ({:.3f})".format(w, s) for s, w in sorted_words[:20]]))
        print(", ".join(["{} ({:.3f})".format(w, s) for s, w in sorted_words[-20:]]))
        print()

In [5]:
def counter_to_vector(counter,size,reverse):
    vector = numpy.zeros(size)
    for word in counter.keys():
        ## look up the integer ID for the string
        word_id = reverse[word]
#         print (counter[word])
        vector[word_id] = counter[word]
    
    return vector / vector.sum()

In [130]:
songkl("Tyler,")

Song: Bastard
Album: Bastard
fucking (0.023), to (0.023), mother (0.020), so (0.019), evil (0.015), I'm (0.014), never (0.014), sessions (0.011), Inside (0.011), My (0.011), inside (0.011), This (0.010), father's (0.010), father (0.010), but (0.009), Sarah (0.009), meet (0.009), tell (0.009), umm (0.009), Well (0.008)
no (-0.001), don't (-0.002), be (-0.002), out (-0.002), we (-0.002), get (-0.002), time (-0.002), some (-0.002), bitch (-0.002), got (-0.003), my (-0.003), And (-0.003), your (-0.003), a (-0.003), on (-0.003), and (-0.004), in (-0.004), the (-0.005), that (-0.005), you (-0.007)

Song: Seven
Album: Bastard
fucking (0.057), I'm (0.037), Fuck (0.035), probably (0.031), like (0.026), my (0.024), relaxing (0.023), Wasted (0.023), Nazis (0.023), Liar (0.023), Hamptons (0.023), from (0.019), youth (0.019), listening (0.019), and (0.019), fans (0.018), with (0.018), um (0.017), everybody (0.016), some (0.016)
so (-0.001), But (-0.001), it's (-0.002), for (-0.002), just (-0.002), 

/Users/jeffrey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [131]:
songkl("Frank")

Song: Street Fighter
Album: nostalgia, ULTRA.
the (nan), I (nan), you (nan), a (nan), to (nan), me (nan), my (nan), in (nan), and (nan), on (nan), it (nan), that (nan), I'm (nan), of (nan), know (nan), for (nan), like (nan), be (nan), no (nan), your (nan)
ships (nan), Buildings (nan), tall (nan), Chloe (nan), shh (nan), Best (nan), clear (nan), messy (nan), any (nan), talents (nan), theremin (nan), awesome (nan), actually (nan), Dill (nan), sits (nan), Sage (nan), Elsesser (nan), skating (nan), handstand (nan), thousand (nan)

Song: Strawberry Swing
Album: nostalgia, ULTRA.
loved (0.140), Without (0.106), should (0.083), blue (0.083), s (0.079), times (0.078), waste (0.074), don (0.074), sky (0.069), t (0.065), could (0.065), here (0.058), swing (0.056), goodbye (0.056), mind (0.046), be (0.046), moment (0.044), good (0.043), cry (0.042), are (0.040)
go (0.005), on (0.005), off (0.004), know (0.004), We (0.004), now (0.003), so (0.003), and (0.003), just (0.002), that (0.001), But (0.0

/Users/jeffrey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [132]:
songkl("Childish")

Song: Difference
Album: Culdesac
different (0.093), am (0.046), just (0.041), write (0.025), I (0.024), her (0.022), city (0.022), Gotta (0.022), use (0.020), always (0.019), have (0.016), mouth (0.016), all (0.015), than (0.014), I've (0.013), watch (0.013), whiter (0.013), want's (0.013), vitamins (0.013), victorious (0.013)
some (-0.002), of (-0.002), girl (-0.002), don't (-0.002), love (-0.002), But (-0.002), with (-0.002), she (-0.002), so (-0.003), do (-0.003), be (-0.003), your (-0.003), like (-0.003), And (-0.003), we (-0.003), up (-0.004), that (-0.004), in (-0.005), the (-0.007), you (-0.013)

Song: Hero
Album: Culdesac
hero (0.208), I'm (0.193), a (0.082), Yeah (0.059), yes (0.055), going (0.046), stop (0.044), top (0.042), won't (0.041), hard (0.037), on (0.037), no (0.024), this (0.018), She (0.017), would (0.016), laying (0.015), jet (0.015), hoping (0.015), MJ (0.015), spent (0.014)
she (-0.002), You (-0.002), was (-0.002), got (-0.003), your (-0.003), I (-0.003), like (

/Users/jeffrey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [133]:
songkl("Kendrick")

Song: The Heart Pt. 2
Album: Overly Dedicated
keeps (0.023), to (0.016), t (0.015), our (0.015), just (0.014), Kendrick (0.014), doing (0.013), bygones (0.012), EP (0.012), Lamar (0.011), mastermind (0.011), beefing (0.011), OD (0.011), probably (0.011), don (0.010), Just (0.010), fucking (0.010), ve (0.010), barely (0.010), of (0.009)
shit (-0.001), it's (-0.001), fuck (-0.002), do (-0.002), nigga (-0.002), can (-0.002), up (-0.002), was (-0.002), be (-0.002), don't (-0.002), is (-0.002), your (-0.004), for (-0.004), that (-0.004), on (-0.004), a (-0.005), and (-0.005), it (-0.005), me (-0.007), you (-0.011)

Song: Growing Apart (To Get Closer)
Album: Overly Dedicated
are (0.119), we (0.118), Where (0.095), going (0.091), slowing (0.069), growing (0.069), should (0.060), to (0.056), following (0.041), s (0.040), closer (0.039), but (0.037), re (0.035), now (0.034), m (0.032), follow (0.031), don (0.027), place (0.026), down (0.026), Tryna (0.022)
wanna (-0.000), tell (-0.000), But (-0

In [136]:
songkl("Vince")

Song: Ramona Park Legend Pt. 1
Album: Summertime ’06
the (nan), I (nan), me (nan), you (nan), a (nan), my (nan), to (nan), up (nan), I'm (nan), on (nan), in (nan), it (nan), be (nan), that (nan), and (nan), no (nan), yeah (nan), know (nan), we (nan), right (nan)
Calling (nan), weak (nan), karma (nan), Family (nan), Hungarians (nan), church (nan), age (nan), Should've (nan), Woulda (nan), useful (nan), pulpit (nan), truthful (nan), Rich (nan), poor (nan), use (nan), fed (nan), piece (nan), Y'all (nan), stick (nan), radios (nan)

Song: Lift Me Up
Album: Summertime ’06
lift (0.344), me (0.249), up (0.237), Lift (0.069), difference (0.033), just (0.030), breathe (0.025), what's (0.024), this (0.022), like (0.021), seems (0.020), presidents (0.020), potions (0.020), pills (0.020), See (0.020), Moses (0.020), Jehovah (0.020), pray (0.019), they (0.019), tell (0.019)
shit (-0.001), bitch (-0.001), got (-0.001), all (-0.001), We (-0.001), okay (-0.002), nigga (-0.002), back (-0.002), your (-0.

/Users/jeffrey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [137]:
songkl("A$AP")

Song: Palace
Album: LIVE.LOVE.A$AP
trill (0.111), this (0.088), is (0.069), how (0.065), real (0.064), cold (0.060), ignorance (0.058), bliss (0.058), Goddamn (0.058), Coast (0.054), then (0.052), East (0.051), give (0.051), nigga (0.044), Still (0.039), wickity (0.038), stoned (0.038), stone (0.038), rowdy (0.038), Stone (0.038)
I'm (0.001), do (0.000), em (0.000), in (-0.001), bitch (-0.001), to (-0.001), back (-0.001), love (-0.001), fuck (-0.002), for (-0.002), got (-0.003), all (-0.003), get (-0.003), me (-0.004), that (-0.006), on (-0.006), you (-0.007), and (-0.007), I (-0.014), the (-0.015)

Song: Peso
Album: LIVE.LOVE.A$AP
pesos (0.128), our (0.124), gunshot (0.098), Gun (0.098), we (0.095), lick (0.092), cock (0.087), Cause (0.086), way (0.081), every (0.077), day (0.076), gonna (0.066), boy (0.063), goes (0.057), is (0.055), roll (0.038), This (0.034), degrees (0.033), told (0.032), show (0.032)
them (0.000), for (0.000), ain't (-0.000), now (-0.000), they (-0.001), be (-0.0

 # end here^

In [38]:
for song_id, song in enumerate(songs):
    # `nonzero` returns a tuple with one element per dimension of the input array
    nzs = numpy.nonzero(song["Vector"])[0]
    kl_terms = song["Vector"][nzs] * numpy.log2(song["Vector"][nzs] / all_vector[nzs])
    
    vocabulary_subset = [vocabulary[i] for i in nzs]

    sorted_words = sorted(zip(kl_terms, vocabulary_subset), reverse=True)
    print(song_id, song["title"])
    print(", ".join(["{} ({:.3f})".format(w, s) for s, w in sorted_words[:20]]))
    print(", ".join(["{} ({:.3f})".format(w, s) for s, w in sorted_words[-20:]]))
    print()
#more frequent in general*multiplied high ratio may not be as important*because it occurs more
#Four Context
#my changes over time
# self referential
# introspective 
#story

0 EARFQUAKE
my (0.131), real (0.093), quake (0.082), it's (0.074), huh (0.073), earth (0.073), fault (0.070), make (0.052), For (0.045), no (0.041), Cause (0.041), Don't (0.038), leave (0.033), heart (0.033), break (0.033), Riding (0.033), And (0.032), want (0.030), making (0.029), shakin (0.024)
cause (0.001), ain't (0.001), fuck (0.001), like (0.000), down (0.000), know (-0.001), not (-0.001), it (-0.001), with (-0.003), on (-0.004), is (-0.004), I'm (-0.004), to (-0.006), in (-0.006), your (-0.007), You (-0.007), a (-0.008), me (-0.008), the (-0.008), I (-0.015)

1 I THINK
Four (0.079), Skate (0.055), real (0.051), I (0.049), tell (0.039), think (0.037), How (0.037), four (0.037), it's (0.036), my (0.035), can (0.029), skate (0.029), quake (0.026), huh (0.024), earth (0.024), love (0.023), fault (0.022), you (0.016), fallen (0.016), This (0.015)
puppet (-0.001), again (-0.001), But (-0.002), like (-0.002), off (-0.002), down (-0.003), with (-0.003), of (-0.003), it (-0.003), and (-0

In [6]:
# song level or album level?

songkl("A$AP")

Song: Palace
Album: LIVE.LOVE.A$AP
trill (0.111), this (0.088), is (0.069), how (0.065), real (0.064), cold (0.060), ignorance (0.058), bliss (0.058), Goddamn (0.058), Coast (0.054), then (0.052), East (0.051), give (0.051), nigga (0.044), Still (0.039), wickity (0.038), stoned (0.038), stone (0.038), rowdy (0.038), Stone (0.038)
I'm (0.001), do (0.000), em (0.000), in (-0.001), bitch (-0.001), to (-0.001), back (-0.001), love (-0.001), fuck (-0.002), for (-0.002), got (-0.003), all (-0.003), get (-0.003), me (-0.004), that (-0.006), on (-0.006), you (-0.007), and (-0.007), I (-0.014), the (-0.015)

Song: Peso
Album: LIVE.LOVE.A$AP
pesos (0.128), our (0.124), gunshot (0.098), Gun (0.098), we (0.095), lick (0.092), cock (0.087), Cause (0.086), way (0.081), every (0.077), day (0.076), gonna (0.066), boy (0.063), goes (0.057), is (0.055), roll (0.038), This (0.034), degrees (0.033), told (0.032), show (0.032)
them (0.000), for (0.000), ain't (-0.000), now (-0.000), they (-0.001), be (-0.0